In [2]:
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest

import time

from src.vsa_encoding import *
from src.estimate_poses import *

np.random.seed(42)



df = pd.read_csv('./data/train_tracked_poses_m.csv')

detections, aspect_thresh, time_thresh = build_encoding_df(df, bins=5, aspect_thresh=None, time_thresh=None)

features = [c for c in detections.columns if 'bin' in c or c in ['position', 'leg_posture']]

detections = detections[features]

# Define the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features)
    ],
    remainder='passthrough'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('isolation_forest', IsolationForest(contamination=0.05, random_state=42))])

# Fit the pipeline to the data
pipeline.fit(detections)

node_counts = [est.tree_.node_count for est in pipeline.named_steps['isolation_forest'].estimators_]
total_nodes = sum(node_counts)

print("Nodes per tree:", node_counts[:5])  # first 5 trees
print("Total number of nodes:", total_nodes)

start = time.time()

video_dir = "./Avenue Dataset/testing_videos"
#estimate_test_vids(video_dir)

test = pd.read_csv('./data/test_tracked_poses_m.csv')
test_detections, _, _ = build_encoding_df(test, aspect_thresh=aspect_thresh, time_thresh=time_thresh)


results = pd.DataFrame({
    'video': test_detections['video'],
    'personID': test_detections['personID'],
    'frameID': test_detections['frameID'],
    'AnomalyScore': pipeline.named_steps["isolation_forest"].decision_function(pipeline.named_steps["preprocessor"].transform(test_detections[features]))
})

results['AnomalyScore'] = 1-results['AnomalyScore']  # Invert scores so higher means more anomalous

end = time.time()
print(f"Processed in {end - start:.2f} seconds")

print(f"Efficiency: {1000*((end-start)/len(test_detections.groupby(['video', 'frameID'])))}ms per frame")

results.to_csv(f'data/results/results_isoforest.csv', index=False)

/Users/joelewis/Desktop/Dissertation/VSA-Human-VAD/src/vsa_encoding.py:212: RuntimeWarning: invalid value encountered in divide
  cos_theta = np.sum(ba * bc, axis=1) / (np.linalg.norm(ba, axis=1) * np.linalg.norm(bc, axis=1))
/Users/joelewis/Desktop/Dissertation/VSA-Human-VAD/src/vsa_encoding.py:244: RuntimeWarning: All-NaN axis encountered
  knee_angle = np.nanmax([l_angle, r_angle], axis=0)


Nodes per tree: [75, 51, 61, 35, 75]
Total number of nodes: 6486


/Users/joelewis/Desktop/Dissertation/VSA-Human-VAD/src/vsa_encoding.py:212: RuntimeWarning: invalid value encountered in divide
  cos_theta = np.sum(ba * bc, axis=1) / (np.linalg.norm(ba, axis=1) * np.linalg.norm(bc, axis=1))
/Users/joelewis/Desktop/Dissertation/VSA-Human-VAD/src/vsa_encoding.py:244: RuntimeWarning: All-NaN axis encountered
  knee_angle = np.nanmax([l_angle, r_angle], axis=0)


Processed in 0.27 seconds
Efficiency: 0.01755658995665408ms per frame
